In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

import tracker
import stats
import metrics
import visualizer
import utils

import importlib

In [ ]:
videofile = "SchmidtDataset/video_P001.mp4"

# Visualize the input
cap = cv2.VideoCapture(videofile)

# Get the first frame
ret, frame = cap.read()

plt.imshow(frame)

## Experiment 2

In [ ]:
# prediction = "SchmidtDataset/video_P001_predicted.csv"
# groundtruth = "SchmidtDataset/video_P001_corrected.csv"

# prediction = "SchmidtDataset/video_P002_predicted.csv"
# groundtruth = "SchmidtDataset/video_P002_corrected.csv"

prediction = "SchmidtDataset/video_P003_predicted.csv"
groundtruth = "SchmidtDataset/video_P003_corrected.csv"


# Load as Pandas DataFrame
gt_src = utils.loadDataFrame(groundtruth)

# Drop duplicates, interpolate, and filter sperm
gt_dd = utils.dropDuplicates(gt_src)

gt = utils.interpolateTracks(gt_dd)

gt = metrics.filterSperm(gt)

In [ ]:
# Make versions of the DataFrames with only the 2 seconds (18 frames) of each sperm
gt_2s = gt[gt['frame'] < 18]

In [ ]:
# Compute stats for each file
ps = 1/1.0476   # pixel size in micrometers
fps = 9    # frames per second
gt = stats.computeAllStats(gt,fps=fps,pixel_size=ps,interpolate=False)

gt_2s = stats.computeAllStats(gt_2s,fps=fps,pixel_size=ps,interpolate=False)

In [ ]:
# Compute earth movers distance between distributions
import pandas as pd
from scipy.stats import wasserstein_distance

emd_df = pd.DataFrame({"Method": ["GT_2s"], "VAP": [None], "VSL": [None], "VCL": [None], "ALH_mean": [None], "ALH_max": [None], "BCF": [None]})

# Compute EMD for each statistic
for stat in ["VAP", "VSL", "VCL", "ALH_mean", "ALH_max", "BCF"]:
    emd_df.loc[0, stat] = wasserstein_distance(gt[stat], gt_2s[stat])

# Print the EMD DataFrame
print(emd_df)

In [ ]:
#print(pred.head())


# Print maxes of each stat for each file as a dataframe
summary_df = pd.DataFrame({
    'Stat': ['VAP', 'VSL', 'VCL', 'ALH_mean', 'ALH_max', 'BCF'],
    'Ground Truth': [gt['VAP'].max(), gt['VSL'].max(), gt['VCL'].max(), gt['ALH_mean'].max(), gt['ALH_max'].max(), gt['BCF'].max()],
    'GT_2s': [gt_2s['VAP'].max(), gt_2s['VSL'].max(), gt_2s['VCL'].max(), gt_2s['ALH_mean'].max(), gt_2s['ALH_max'].max(), gt_2s['BCF'].max()]
})

print(summary_df)

print("Mins:",gt["VAP"].min(), gt["VSL"].min(), gt["VCL"].min(), gt["ALH_mean"].min(), gt["ALH_max"].min(), gt["BCF"].min())


In [ ]:
# Build 3x6 subplot for each stat and each method

gt_vap = gt.groupby('sperm').first()['VAP']
gt_vsl = gt.groupby('sperm').first()['VSL']
gt_vcl = gt.groupby('sperm').first()['VCL']

gt_2s_vap = gt_2s.groupby('sperm').first()['VAP']
gt_2s_vsl = gt_2s.groupby('sperm').first()['VSL']
gt_2s_vcl = gt_2s.groupby('sperm').first()['VCL']

# Create a figure with 2 rows and 3 columns
fig, axs = plt.subplots(2, 3, figsize=(10, 6))

import math
vap_max = math.ceil(max(gt_vap.max(), gt_2s_vap.max()))
vsl_max = math.ceil(max(gt_vsl.max(), gt_2s_vsl.max()))
vcl_max = math.ceil(max(gt_vcl.max(), gt_2s_vcl.max()))

# Create horizontal violin plots for each stat and each method
# axs[0, 0].violinplot(gt_vap, vert=False, showmeans=True)
# axs[0, 0].set_title('Ground Truth VAP')
# axs[1, 0].violinplot(gt_2s_vap, vert=False, showmeans=True)
# axs[1, 0].set_title('GT_2s VAP')
# axs[0, 1].violinplot(gt_vsl, vert=False, showmeans=True)
# axs[0, 1].set_title('Ground Truth VSL')
# axs[1, 1].violinplot(gt_2s_vsl, vert=False, showmeans=True)
# axs[1, 1].set_title('GT_2s VSL')
# axs[0, 2].violinplot(gt_vcl, vert=False, showmeans=True)
# axs[0, 2].set_title('Ground Truth VCL')
# axs[1, 2].violinplot(gt_2s_vcl, vert=False, showmeans=True)
# axs[1, 2].set_title('GT_2s VCL')

# Create bins for each histogram for each stat
vap_bins = np.linspace(0, vap_max, 20)
vsl_bins = np.linspace(0, vsl_max, 20)
vcl_bins = np.linspace(0, vcl_max, 20)

# Create histograms for each stat and each method
axs[1, 0].hist(gt_vap, bins=vap_bins, color='purple', alpha=1.0, density=True)
axs[1, 0].set_title('30 Second Clip VAP')
axs[0, 0].hist(gt_2s_vap, bins=vap_bins, color='purple', alpha=1.0, density=True)
axs[0, 0].set_title('2 Second Clip VAP')
axs[1, 1].hist(gt_vsl, bins=vsl_bins, color='purple', alpha=1.0, density=True)
axs[1, 1].set_title('30 Second Clip VSL')
axs[0, 1].hist(gt_2s_vsl, bins=vsl_bins, color='purple', alpha=1.0, density=True)
axs[0, 1].set_title('2 Second Clip VSL')
axs[1, 2].hist(gt_vcl, bins=vcl_bins, color='purple', alpha=1.0, density=True)
axs[1, 2].set_title('30 Second Clip VCL')
axs[0, 2].hist(gt_2s_vcl, bins=vcl_bins, color='purple', alpha=1.0, density=True)
axs[0, 2].set_title('2 Second Clip VCL')


# Set x-limits for each subplot
axs[0, 0].set_xlim(0, vap_max)
axs[1, 0].set_xlim(0, vap_max)
axs[0, 1].set_xlim(0, vsl_max)
axs[1, 1].set_xlim(0, vsl_max)
axs[0, 2].set_xlim(0, vcl_max)
axs[1, 2].set_xlim(0, vcl_max)

axs[0, 0].set_ylim(0, 0.075)
axs[1, 0].set_ylim(0, 0.075)
axs[0, 1].set_ylim(0, 0.075)
axs[1, 1].set_ylim(0, 0.075)
axs[0, 2].set_ylim(0, 0.075)
axs[1, 2].set_ylim(0, 0.075)


# # Set colors of the bodies
# for ax in axs.flat:
#     for pc in ax.collections:
#         pc.set_facecolor('purple')
#         pc.set_edgecolor('black')
#         pc.set_alpha(0.7)

# Set common axis labels
for ax in axs[:,0]:
    ax.set_ylabel('Probability Density', fontsize=12)
for ax in axs[1,:]:
    ax.set_xlabel('Velocity (µm/s)', fontsize=12)

# Adjust layout
plt.tight_layout()
plt.savefig("experiment2.pdf", dpi=300)
